In [3]:
import requests
import csv

def get_organism_info(accession_numbers, output_csv):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    db = "nuccore"
    rettype = "gb"
    retmode = "text"

    with open(output_csv, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["Accession Number", "Organism Name", "Taxonomy"])

        for accession_number in accession_numbers:
            url = f"{base_url}?db={db}&id={accession_number}&rettype={rettype}&retmode={retmode}"
            response = requests.get(url)

            if response.status_code == 200:
                record_lines = response.text.splitlines()
                organism_line = next((line for line in record_lines if "ORGANISM" in line), None)

                if organism_line:
                    organism_name = organism_line.split("  ")[-1].strip()
                    taxonomy_lines = [line.strip() for line in record_lines if line.startswith("            ")]
                    taxonomy = " ".join(taxonomy_lines)

                    csv_writer.writerow([accession_number, organism_name, taxonomy])
                else:
                    csv_writer.writerow([accession_number, "Organism name not found", "Taxonomy not found"])

            else:
                csv_writer.writerow([accession_number, f"Error fetching data. Status code: {response.status_code}", ""])

if __name__ == "__main__":
    # Replace "your_accession_numbers_file.txt" and "output_organism_info.csv" with the actual file paths
    with open("/path/to/SequenceList.txt", "r") as file:
        accession_numbers = [line.strip() for line in file]

    get_organism_info(accession_numbers, "/path/to/output.csv")
